# Project Corona
## Philosophy and History of Science with Computational Means
Prof Dr. Gerd Graßhoff
### Filter dataframe, NLTP

# Libraries, data

In [20]:
import pandas as pd
import spacy
import re

In [21]:
# Import English library
nlp = spacy.load('en_core_web_sm')

In [48]:
dfRem=pd.read_json("dfRemdesivierResearchObjects.json")

In [49]:
len(dfRem)

2180

In [50]:
dfRem.head(2)

,PMID,title,abstract
0,32448345.0,Evaluation of the efficacy and safety of intra...,BACKGROUND: Coronavirus disease 2019 (COVID-19...
1,32370766.0,Role of adjunctive treatment strategies in COV...,The coronavirus disease (COVID-19) pandemic ha...


# Filter

## selecting rows

In [60]:
def filter(zeile):
    cond=zeile["title"].str.contains("disease",na=False)
    return(cond)

In [61]:
df=dfRem[filter]
df.head(2)

,PMID,title,abstract
5,32169119.0,"The origin, transmission and clinical therapie...","An acute respiratory disease, caused by a nove..."
12,32435920.0,Lithium’s antiviral effects: a potential drug ...,BACKGROUND: Since its introduction in modern m...
